In [8]:
!pip install faiss-cpu

     ---------------------------------------- 10.8/10.8 MB 3.4 MB/s eta 0:00:00


In [154]:
#importing all necessary libraries
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import faiss
from sklearn.preprocessing import StandardScaler

In [155]:
# Load CSV file but only taking 20 rows as my computer is not able to process all text
csv_file_path = 'bigBasketProducts.csv'  # Replace with your CSV file path
df = pd.read_csv(csv_file_path).loc[:20]

In [203]:
data.dropna(inplace=True)
df.head(20)

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,embedding
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.00,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...,"[0.3630827, -0.12896645, 1.2466044, 0.08737200..."
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.00,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ...","[-0.12922071, 0.3432063, 1.7577443, 0.1186419,..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.00,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m...","[0.21217324, -0.14305389, 0.9560878, -0.375539..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.00,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...,"[-0.75246495, 0.8200897, 0.97320503, 0.3238592..."
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.00,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,"[-0.15321632, -0.35328963, 1.3477589, 0.203393..."
5,6,Germ - Removal Multipurpose Wipes,Cleaning & Household,All Purpose Cleaners,Nature Protect,169.00,199.0,Disinfectant Spray & Cleaners,3.3,Stay protected from contamination with Multipu...,"[-0.3405039, 0.3087283, 1.4126767, 0.3522587, ..."
6,7,Multani Mati,Beauty & Hygiene,Skin Care,Satinance,58.00,58.0,Face Care,3.6,Satinance multani matti is an excellent skin t...,"[0.1631037, -0.7672594, 0.9340905, 0.44764805,..."
7,8,Hand Sanitizer - 70% Alcohol Base,Beauty & Hygiene,Bath & Hand Wash,Bionova,250.00,250.0,Hand Wash & Sanitizers,4.0,70%Alcohol based is gentle of hand leaves skin...,"[-1.2456663, -0.24894762, 1.2057824, 0.0379022..."
8,9,Biotin & Collagen Volumizing Hair Shampoo + Bi...,Beauty & Hygiene,Hair Care,StBotanica,1098.00,1098.0,Shampoo & Conditioner,3.5,"An exclusive blend with Vitamin B7 Biotin, Hyd...","[-0.15945518, 0.07644405, 2.2635715, 0.4166354..."
9,10,"Scrub Pad - Anti- Bacterial, Regular",Cleaning & Household,"Mops, Brushes & Scrubs",Scotch brite,20.00,20.0,"Utensil Scrub-Pad, Glove",4.3,Scotch Brite Anti- Bacterial Scrub Pad thoroug...,"[-0.13428208, 0.40889066, 0.892387, 0.6890382,..."


In [157]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [158]:
# Tokenize and encode text using the language model
def encode_text(text):
    tokens = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

In [159]:
# Apply the encoding function to each row in the CSV file
df['embedding'] = df['description'].apply(encode_text)

In [160]:
#  Normalize the embeddings
scaler = StandardScaler()
df['embedding'] = df['embedding'].apply(lambda x: scaler.fit_transform(x.reshape(-1, 1)).flatten())

In [161]:
# Store embeddings in a Faiss index
d = len(df['embedding'][0])  # Dimensionality of embeddings
index = faiss.IndexFlatL2(d)
embeddings = np.vstack(df['embedding'].to_numpy())
index.add(embeddings.astype(np.float32))



In [162]:
d

768

In [163]:

def query_llm(query_text, k=5):
    query_embedding = encode_text(query_text)
    normalized_query = scaler.transform(query_embedding.reshape(-1, 1)).flatten()
    
    _, indices = index.search(np.array([normalized_query], dtype=np.float32), k)
    return df.iloc[indices[0]]['description'].tolist()

In [190]:
# query-1
query_result = query_llm("Give description of  Sapphire Butter cookies", k=1)
print("Query Result:", query_result)

Query Result: ['Enjoy a tin full of delicious butter cookies made with pure Danish recipe. Enjoy them as an energy snack. or with tea and coffee. Sapphire Butter cookies make a perfect gift pack for your loved ones. This product is 100% vegetarian.']


In [181]:
#query-2
query_result = query_llm("About Oxy face wash", k=1)
print("Query Result:", query_result)

Query Result: ['This face wash deeply cleanses dirt and impurities. Active ingredients help remove excess oil and other pimple causing germs, leaving skin clean and fresh. Its a oil-control & pimple clearing ingredients which rid your skin of excess oil and other a pimple-causing germs, leaving it clean and fresh.']


In [201]:
# query-3
query_result = query_llm("About Wheatgrass", k=1)
print("Query Result:", query_result)

Query Result: ['Wheatgrass is a superfood potent health food with amazing benefits. It contains all minerals, Vitamins A, B- complex, C, E, I and K. It is extremely rich in protein and contains 17 amino acids. It is thought to benefit health only when fresh and taken on an empty stomach immediately after being mixed.']
